In [1]:
import pandas as pd 
import urllib
from sqlalchemy import create_engine 
import sqlalchemy as db
import pyodbc 
from sqlalchemy.sql import text
from openpyxl import load_workbook
import pytz 
from datetime import datetime
from pathlib import Path
import os

In [2]:
latest_update_time  = ''
current_date = datetime.now().strftime('%Y-%m-%d')
date = str(datetime.now())
## function to create a connection to the db 
def make_connection():
    server = urllib.parse.quote_plus(r'Driver={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.0.so.1.1};SERVER=10.197.194.138,1433;DATABASE=AnalyticsDB;UID=sa;PWD=Admin@123')
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(server))
    print(date + ": Connection established")
    return engine

In [3]:
def fetch_latest_time(connect):
    query = ('select opentime = max(opentime )from  sm9_incidents as s join incident_dashboard as s1 on s.incidentid = s1.incidentid')
    result = connect.execute(text(query))
    for r in result:
        latest_update_time = r['opentime']
    return latest_update_time

In [4]:
def fetch_latest_incident(connect):
    query = ("select * from [10.168.29.147, 51001].[TREP].[dbo].[VW_AOC_SM9_INM_Updates] where OpenTime >  '{}'".format(latest_update_time))
    df = pd.read_sql_query(query, connect)
    return df 

In [5]:
# def write_result(dataframe, filename, sheetname):
#     if dataframe.empty:
#         print("File is not written as no incident has been opened")
#         #exit()
#     else:
#         config = Path(filename)
        
#         if config.is_file():
#             book = load_workbook(filename)
#             writer = pd.ExcelWriter(filename, engine='openpyxl')
#             writer.book = book
#             dataframe.to_excel(writer, sheet_name=sheetname)
#             writer.save()
#             writer.close()
#         else:
#             dataframe.to_excel(filename, sheet_name=sheetname)
#         print("Data got downloaded at location ", filename)

In [6]:
if __name__ == '__main__':
    if latest_update_time is '':
        conn = make_connection()
        connect = conn.connect()
        latest_update_time = fetch_latest_time(connect)
        #query = ("select * from [dbo].[VW_AOC_SM9_INM_Updates] where opentime >= {}".format(current_time))
        query = ("select * from [10.168.29.147, 51001].[TREP].[dbo].[VW_AOC_SM9_INM_Updates] where opentime >'{}'".format(latest_update_time))
        df = pd.read_sql_query(query, connect)
        print("The shape of the data downloaded ", df.shape)
        df.to_excel(r"/home/Notebook_Scripts/ML_MODEL_2020/Input/Hans_"+current_date+"_data.xlsx")
#        filename = (r"/home/Notebook_Scripts/ML_MODEL_2020/Input/Hans_"+current_date+"_data.xlsx")
#         print(filename)
#         config = Path(filename) 
#         sheet_count = 0
#         if config.is_file():
#             sheetname  = "Sheet " + str(sheet_count)
#             sheet_count += 1
#         else:
#             sheetname="Sheet " + str(sheet_count)
#             sheet_count +=1
#         write_result(df, filename, sheetname)
        
#     else:
#         conn = make_connection()
#         result = fetch_latest_incident(conn)
#         print(result.head(5))
#         print(result.shape)
#         filename = (r"/home/Notebook_Scripts/ML_MODEL_2020/Input/Hans_"+current_date+"_data.xlsx")
#         config = Path(filename) 
#         sheet_count = 1
#         if config.is_file():
#             sheetname  = "Sheet 1" + str(sheet_count)
#             sheet_count += 1
#         else:
#             sheetname="Sheet " + str(sheet_count)
#             sheet_count +=1
#         write_result(result, filename, sheetname)
        latest_update_time = fetch_latest_time(conn)
    print("The last ticket was open on ", latest_update_time)
#     print(sheetname)
#print(current_date)   
%run ./Events_Model_Testing.ipynb

2020-09-16 17:50:05.903514: Connection established
The shape of the data downloaded  (32, 15)
The last ticket was open on  2020-09-16 10:07:04
Classifier LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)
(32, 15)
Sub Categories  has been updated
2020-09-16 17:50:16.865566: Connection established
Data inserted to the DB (32, 3)


FileNotFoundError: [Errno 2] No such file or directory: '/home/Dailydata/Hans_2020-09-16 17:data.xlsx'